# Laboratorio 6

## Integrantes

### Sergio Orellana - 221122

### Andre Marroquin - 22266

### Rodrigo Mansilla - 22611

# Link del repositorio

https://github.com/mar22266/LABORATORIOS-IA.git

# Link del video

# Tasks 1 - Teoría 

1. En un juego de suma cero para dos jugadores, ¿cómo funciona el algoritmo minimax para determinar la 
estrategia óptima para cada jugador? ¿Puede explicarnos el concepto de "valor minimax" y su importancia 
en este contexto? 

        R//


2. Compare y contraste el algoritmo minimax con la poda alfa-beta. ¿Cómo mejora la poda alfa-beta la 
eficiencia del algoritmo minimax, particularmente en árboles de caza grandes? Proporcione un ejemplo 
para ilustrar la diferencia en la complejidad computacional entre la poda minimax y alfa-beta. 

        R//


3. ¿Cuál es el papel de expectiminimax en juegos con incertidumbre, como aquellos que involucran nodos de 
azar o información oculta? ¿En qué se diferencia el expectiminimax del minimax en el manejo de 
resultados probabilísticos y cuáles son los desafíos clave que aborda? 

        R//

        

# Task 2 - Connect Four

In [16]:
import math
import random
import copy

# Dimensiones del tablero
ROW_COUNT = 6
COLUMN_COUNT = 7

# Definición de piezas: 0 vacío, 1 y 2 jugadores
EMPTY = 0

# Crea el tablero vacío
def create_board():
    # Retorna una matriz 6x7 con ceros
    board = [[EMPTY for _ in range(COLUMN_COUNT)] for _ in range(ROW_COUNT)]
    return board

# Coloca la pieza en el tablero en row, col
def drop_piece(board, row, col, piece):
    board[row][col] = piece

# Verifica si la columna tiene espacio
def is_valid_location(board, col):
    return board[0][col] == EMPTY

# Retorna la fila más baja libre en la columna
def get_next_open_row(board, col):
    for r in range(ROW_COUNT - 1, -1, -1):
        if board[r][col] == EMPTY:
            return r
    return None

# Revisa si hay 4 en línea para la pieza
def winning_move(board, piece):
    # Horizontal
    for r in range(ROW_COUNT):
        for c in range(COLUMN_COUNT - 3):
            if board[r][c] == piece and board[r][c+1] == piece and board[r][c+2] == piece and board[r][c+3] == piece:
                return True
    # Vertical
    for c in range(COLUMN_COUNT):
        for r in range(ROW_COUNT - 3):
            if board[r][c] == piece and board[r+1][c] == piece and board[r+2][c] == piece and board[r+3][c] == piece:
                return True
    # Diagonal positiva
    for r in range(3, ROW_COUNT):
        for c in range(COLUMN_COUNT - 3):
            if board[r][c] == piece and board[r-1][c+1] == piece and board[r-2][c+2] == piece and board[r-3][c+3] == piece:
                return True
    # Diagonal negativa
    for r in range(ROW_COUNT - 3):
        for c in range(COLUMN_COUNT - 3):
            if board[r][c] == piece and board[r+1][c+1] == piece and board[r+2][c+2] == piece and board[r+3][c+3] == piece:
                return True
    return False

# Retorna la lista de posiciones que conforman la línea ganadora
def get_winning_positions(board, piece):
    for r in range(ROW_COUNT):
        for c in range(COLUMN_COUNT - 3):
            if board[r][c] == piece and board[r][c+1] == piece and board[r][c+2] == piece and board[r][c+3] == piece:
                return [(r, c+i) for i in range(4)]
    for c in range(COLUMN_COUNT):
        for r in range(ROW_COUNT - 3):
            if board[r][c] == piece and board[r+1][c] == piece and board[r+2][c] == piece and board[r+3][c] == piece:
                return [(r+i, c) for i in range(4)]
    for r in range(3, ROW_COUNT):
        for c in range(COLUMN_COUNT - 3):
            if board[r][c] == piece and board[r-1][c+1] == piece and board[r-2][c+2] == piece and board[r-3][c+3] == piece:
                return [(r-i, c+i) for i in range(4)]
    for r in range(ROW_COUNT - 3):
        for c in range(COLUMN_COUNT - 3):
            if board[r][c] == piece and board[r+1][c+1] == piece and board[r+2][c+2] == piece and board[r+3][c+3] == piece:
                return [(r+i, c+i) for i in range(4)]
    return []

# Retorna las columnas disponibles para jugar
def get_valid_locations(board):
    valid_locations = []
    for col in range(COLUMN_COUNT):
        if is_valid_location(board, col):
            valid_locations.append(col)
    return valid_locations

# Verifica si el juego terminó
def is_terminal_node(board):
    return winning_move(board, 1) or winning_move(board, 2) or len(get_valid_locations(board)) == 0

# Evalúa una ventana de 4 celdas
def evaluate_window(window, piece):
    score = 0
    opponent_piece = 1 if piece == 2 else 2
    if window.count(piece) == 4:
        score += 100
    elif window.count(piece) == 3 and window.count(EMPTY) == 1:
        score += 5
    elif window.count(piece) == 2 and window.count(EMPTY) == 2:
        score += 2
    if window.count(opponent_piece) == 3 and window.count(EMPTY) == 1:
        score -= 4
    return score

# Evaluación heurística del tablero
def score_position(board, piece):
    score = 0
    center_column = [board[r][COLUMN_COUNT // 2] for r in range(ROW_COUNT)]
    score += center_column.count(piece) * 3
    for r in range(ROW_COUNT):
        row_array = board[r]
        for c in range(COLUMN_COUNT - 3):
            window = row_array[c:c+4]
            score += evaluate_window(window, piece)
    for c in range(COLUMN_COUNT):
        col_array = [board[r][c] for r in range(ROW_COUNT)]
        for r in range(ROW_COUNT - 3):
            window = col_array[r:r+4]
            score += evaluate_window(window, piece)
    for r in range(ROW_COUNT - 3):
        for c in range(COLUMN_COUNT - 3):
            window = [board[r+i][c+i] for i in range(4)]
            score += evaluate_window(window, piece)
    for r in range(3, ROW_COUNT):
        for c in range(COLUMN_COUNT - 3):
            window = [board[r-i][c+i] for i in range(4)]
            score += evaluate_window(window, piece)
    return score

# Algoritmo minimax con o sin poda alfa-beta
def minimax(board, depth, alpha, beta, maximizingPlayer, piece, use_pruning):
    valid_locations = get_valid_locations(board)
    terminal = is_terminal_node(board)
    opponent_piece = 1 if piece == 2 else 2
    if depth == 0 or terminal:
        if terminal:
            if winning_move(board, piece):
                return (None, math.inf)
            elif winning_move(board, opponent_piece):
                return (None, -math.inf)
            else:
                return (None, 0)
        else:
            return (None, score_position(board, piece))
    if maximizingPlayer:
        value = -math.inf
        best_col = random.choice(valid_locations)
        for col in valid_locations:
            row = get_next_open_row(board, col)
            b_copy = copy.deepcopy(board)
            drop_piece(b_copy, row, col, piece)
            new_score = minimax(b_copy, depth-1, alpha, beta, False, opponent_piece, use_pruning)[1]
            if new_score > value:
                value = new_score
                best_col = col
            if use_pruning:
                alpha = max(alpha, value)
                if alpha >= beta:
                    break
        return best_col, value
    else:
        value = math.inf
        best_col = random.choice(valid_locations)
        for col in valid_locations:
            row = get_next_open_row(board, col)
            b_copy = copy.deepcopy(board)
            drop_piece(b_copy, row, col, piece)
            new_score = minimax(b_copy, depth-1, alpha, beta, True, opponent_piece, use_pruning)[1]
            if new_score < value:
                value = new_score
                best_col = col
            if use_pruning:
                beta = min(beta, value)
                if alpha >= beta:
                    break
        return best_col, value

# Imprime el tablero; resalta las posiciones ganadoras en rojo
def print_board(board, winning_positions=[]):
    for r in range(ROW_COUNT):
        row_str = ""
        for c in range(COLUMN_COUNT):
            cell = board[r][c]
            if (r, c) in winning_positions:
                row_str += "\033[91m" + str(cell) + "\033[0m" + " "
            else:
                row_str += str(cell) + " "
        print(row_str)
    print("")  # Se quitan los índices de columna

# Función principal de juego
def iniciar_juego(mode='human_vs_ai', depth=4):
    board = create_board()
    game_over = False
    turn = 0
    print_board(board)
    while not game_over:
        if mode == 'human_vs_ai':
            if turn == 0:
                valid_cols = get_valid_locations(board)
                col = -1
                while col not in valid_cols:
                    try:
                        col = int(input(f"Col (0-{COLUMN_COUNT-1}): "))
                        if col not in valid_cols:
                            print("No válido, intente otra vez.")
                    except ValueError:
                        print("Entrada inválida.")
                row = get_next_open_row(board, col)
                drop_piece(board, row, col, 1)
                if winning_move(board, 1):
                    winning_pos = get_winning_positions(board, 1)
                    print_board(board, winning_pos)
                    print("¡Ganaste!")
                    game_over = True
                else:
                    print_board(board)
            else:
                print("IA pensando...")
                use_pruning = True
                col, score = minimax(board, depth, -math.inf, math.inf, True, 2, use_pruning)
                if col is None:
                    print("Empate!")
                    game_over = True
                else:
                    row = get_next_open_row(board, col)
                    drop_piece(board, row, col, 2)
                    if winning_move(board, 2):
                        winning_pos = get_winning_positions(board, 2)
                        print_board(board, winning_pos)
                        print("IA gana.")
                        game_over = True
                    else:
                        print_board(board)
            turn = (turn + 1) % 2
        elif mode == 'ai_vs_ai':
            if turn == 0:
                print("IA sin poda...")
                use_pruning = False
                col, score = minimax(board, depth, -math.inf, math.inf, True, 1, use_pruning)
                piece = 1
            else:
                print("IA con poda...")
                use_pruning = True
                col, score = minimax(board, depth, -math.inf, math.inf, True, 2, use_pruning)
                piece = 2
            if col is None:
                print("Empate!")
                game_over = True
            else:
                row = get_next_open_row(board, col)
                drop_piece(board, row, col, piece)
                if winning_move(board, piece):
                    winning_pos = get_winning_positions(board, piece)
                    print_board(board, winning_pos)
                    if turn == 0:
                        print("IA sin poda gana!")
                    else:
                        print("IA con poda gana!")
                    game_over = True
                else:
                    print_board(board)
            turn = (turn + 1) % 2
        else:
            print("Modo no válido.")
            break

# Función para iniciar el juego 
def ejecutar():
    print("1. Humano vs IA")
    print("2. IA vs IA")
    mode_input = input("Elija 1 o 2: ")
    if mode_input == '1':
        print("Modo: Humano vs IA")
        iniciar_juego(mode='human_vs_ai', depth=4)
    elif mode_input == '2':
        print("Modo: IA vs IA")
        iniciar_juego(mode='ai_vs_ai', depth=4)
    else:
        print("Opción no válida.")

ejecutar()


1. Humano vs IA
2. IA vs IA
Modo: IA vs IA
0 0 0 0 0 0 0 
0 0 0 0 0 0 0 
0 0 0 0 0 0 0 
0 0 0 0 0 0 0 
0 0 0 0 0 0 0 
0 0 0 0 0 0 0 

IA sin poda...
0 0 0 0 0 0 0 
0 0 0 0 0 0 0 
0 0 0 0 0 0 0 
0 0 0 0 0 0 0 
0 0 0 0 0 0 0 
0 0 0 1 0 0 0 

IA con poda...
0 0 0 0 0 0 0 
0 0 0 0 0 0 0 
0 0 0 0 0 0 0 
0 0 0 0 0 0 0 
0 0 0 0 0 0 0 
0 0 2 1 0 0 0 

IA sin poda...
0 0 0 0 0 0 0 
0 0 0 0 0 0 0 
0 0 0 0 0 0 0 
0 0 0 0 0 0 0 
0 0 1 0 0 0 0 
0 0 2 1 0 0 0 

IA con poda...
0 0 0 0 0 0 0 
0 0 0 0 0 0 0 
0 0 0 0 0 0 0 
0 0 0 0 0 0 0 
0 0 1 2 0 0 0 
0 0 2 1 0 0 0 

IA sin poda...
0 0 0 0 0 0 0 
0 0 0 0 0 0 0 
0 0 0 0 0 0 0 
0 0 1 0 0 0 0 
0 0 1 2 0 0 0 
0 0 2 1 0 0 0 

IA con poda...
0 0 0 0 0 0 0 
0 0 0 0 0 0 0 
0 0 2 0 0 0 0 
0 0 1 0 0 0 0 
0 0 1 2 0 0 0 
0 0 2 1 0 0 0 

IA sin poda...
0 0 0 0 0 0 0 
0 0 0 0 0 0 0 
0 0 2 0 0 0 0 
0 0 1 1 0 0 0 
0 0 1 2 0 0 0 
0 0 2 1 0 0 0 

IA con poda...
0 0 0 0 0 0 0 
0 0 2 0 0 0 0 
0 0 2 0 0 0 0 
0 0 1 1 0 0 0 
0 0 1 2 0 0 0 
0 0 2 1 0 0 0 

IA sin poda...
0 0

# Referencias

El código se generó usando el siguiente prompt:

"ayudame con la validacion de reglas y la implementacion del algoritmo minimax para este codigo connect 4. porfavor"

Con base en este prompt, se utilizó una herramienta de IA generativa (ChatGPT) para obtener una versión base que incluyera:

Validación de reglas y creación del tablero:
Se definió el tablero como una matriz de 6 filas por 7 columnas, se programaron funciones para validar jugadas (por ejemplo, verificar si una columna tiene espacio, obtener la fila disponible y detectar jugadas ganadoras mediante revisiones horizontales, verticales y diagonales).

Implementación del algoritmo minimax:
Se implementó el algoritmo minimax, con la opción de activar o desactivar la poda alfa-beta, siguiendo el esquema habitual del algoritmo para juegos competitivos. Esto incluye la generación de posibles movimientos, la evaluación heurística del tablero y el uso de poda para mejorar la eficiencia en la búsqueda de la jugada óptima.